# Análisis y visualización de datos con python
# 3. Limpieza de datos

La limpieza de datos es el proceso de identificar, corregir o eliminar datos incorrectos, incompletos, irrelevantes o duplicados en un conjunto de datos. La limpieza de datos es importante para garantizar la precisión y la calidad de los datos y evitar errores en el análisis posterior. Algunas técnicas comunes de limpieza de datos incluyen la eliminación de valores atípicos, la eliminación de valores faltantes, la corrección de errores tipográficos y la eliminación de duplicados.

Empezaremos cargando los datos.

In [1]:
import pandas as pd

file_mfc = 'data_raw/Modulo-de-Fosas-Comunes_actualizacion24nov2022_VD.xlsx'
df = pd.read_excel(file_mfc, sheet_name="MFC", nrows=1000, index_col='ID' )
df.head()

,Estado_origen,Municipio_origen,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Rdoc,Marca_temporal,Datos alternativos
ID,,,,,,,,,,,,,,,,,,,
XX-P001,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-30,2018-03-04 00:00:00,NaT,Restos cremados,Masculino,55,Desconocido,NaN,NaN,NaN,,UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA,Sí,2020-03-23,NaN
XX-P002,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26,2018-12-12 00:00:00,NaT,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23,NaN
XX-P003,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26,2018-12-13 00:00:00,NaT,Cadáver,Masculino,NaN,Conocido,Cruz,Lucero,Alberto,Alberto Cruz Lucero,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23,NaN
XX-P004,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26,2018-12-14 00:00:00,NaT,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23,NaN
XX-P005,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26,2018-12-18 00:00:00,NaT,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23,NaN


Una consideración importante es si se modificara la columna original o se generará una columna nueva con los datos limpios. Qué opción se escoge depende de varios factores, cómo el tamaño de la tabla o si nos interesa conservar la información original. En este tutorial crearemos columnas nuevas para poder contrastar los cambios y al final quitaremos las que no nos interesen.

## 3.1 Tipos de datos

Cómo vimos anteriormente hay varios tipos de datos como: texto, númerico, fecha, etc.
De manera automática pandas infiere el tipo de datos al leer el archivo, de una manera muy similar a como Excel interpreta si un dato es texto o numérico.

Los tipos de datos de pandas son:
* `object`: texto o mezcla de varios tipos de datos, este es el tipo de dato por defecto
* `category`: información categórica, similar a Factors en R
* `int64`: números enteros
* `float64`: números con punto decimal
* `bool`: valores verdadero o falso
* `datetime64`: fechas
* `timedelta[ns]`: diferencias de tiempo

Podemos ver los tipos de datos inferidos usando la opción `.dtypes`.

In [2]:
df.dtypes

Estado_origen                   object
Municipio_origen                object
Panteón_origen                  object
Estatus_FC                      object
Fecha_inhumación        datetime64[ns]
Fecha_defunción                 object
Fecha_exhumación        datetime64[ns]
Restos_tipo                     object
Sexo                            object
Edad                            object
Conocido_Desconocido            object
Primer apellido                 object
Segundo Apellido                object
Nombre(s)                       object
Nombre completo                 object
Institución_origen              object
Rdoc                            object
Marca_temporal          datetime64[ns]
Datos alternativos             float64
dtype: object

Dependiendo del tipo de dato se pueden hacer diferentes operaciones y hay tipos de limpieza específica.

Cómo vimos en la definición de datos ordenados o tidy data idealmente cada tabla tendrá solo datos de un tipo. Esto es para facilitar las operaciones. Si esto no es posible es importante que una columna o variable tenga datos de un solo tipo, ya que esto puede generar errores.

Por ejemplo, si tratamos de obtener el promedio de las edades nos genera un error de `ValueError`, ya que hay valores de texto `str` y número `int` mezclados, además de datos faltantes.

In [3]:
df['Edad'].unique()

array([55, nan, '19 semanas', '12 semanas', 31, 18, 39, 35, 74, 26,
       '36 semanas', '15 semanas', 83, 89, 64, 78, 73, 81, 75, 60, 63, 27,
       45, 40, 38, 25, 90, 42, 77, 70, 47, 32, '20 semanas', 76, 84, 87,
       66, 68, 94, 71, 92, 49, 69, 50, 80, 28, 37, 30, 20, '<17',
       '26 semanas', 34, 65, 86, 67, 58, 52, 85, '21 semanas',
       '17.6 semanas', '9 semanas', '33 semanas', 46, 22, '26.8 semanas',
       '22 semanas', 54, 33, 48, 53, 44, 57, 72, 41, 79, '29 semanas', 51,
       82, '67-82', 88, 36, 96, 62, 91, '13 semanas', '5 semanas',
       '11 semanas', '1 mes', '25 semanas', '26-32 semanas', '14 semanas',
       15, '40 semanas', '24 semanas', 24, 29, 13, 6, '23 semanas', 101,
       93, 59, 19, '13.7 semanas', 61, 97, '86-87', 95, 17, 16, 23, 21,
       '18 semanas', 56, '30 semanas', 11, 5, '30-40', 1], dtype=object)

Existen varias formas de fijar el tipo de datos, una de ellas es usar la opción `dtype` al leer el dataframe. Esta opción puede recibir un solo tipo de datos o un diccionarió de tipos de datos.

Por ejemplo, podemos leer toda la tabla cómo texto usando `dtype='str'`, lo cual obligará a ciertos tipos de datos como fechas o números a aparecer cómo texto. Hacer esto es una buena opción con los profiles iniciales cuando hay columnas que no se leen correctamente.

También se puede pasar un diccionario marcando el tipo de datos de cada columna específicamente. Sin embargo, si los datos no están limpios se pueden generar errores de lectura. Esta opción es ideal ya que los datos están limpios

In [4]:
df = pd.read_excel(file_mfc, sheet_name="MFC", nrows=1000, index_col='ID', dtype='str' )
df.head()

,Estado_origen,Municipio_origen,Panteón_origen,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Rdoc,Marca_temporal,Datos alternativos
ID,,,,,,,,,,,,,,,,,,,
XX-P001,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-30 00:00:00,2018-03-04 00:00:00,NaN,Restos cremados,Masculino,55,Desconocido,NaN,NaN,NaN,,UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA,Sí,2020-03-23 00:00:00,NaN
XX-P002,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P003,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-13 00:00:00,NaN,Cadáver,Masculino,NaN,Conocido,Cruz,Lucero,Alberto,Alberto Cruz Lucero,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P004,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P005,Ciudad de México,Miguel Hidalgo,Panteón Civil de Dolores,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN


Los tipos de datos aparecen cómo object

In [5]:
df.dtypes

Estado_origen           object
Municipio_origen        object
Panteón_origen          object
Estatus_FC              object
Fecha_inhumación        object
Fecha_defunción         object
Fecha_exhumación        object
Restos_tipo             object
Sexo                    object
Edad                    object
Conocido_Desconocido    object
Primer apellido         object
Segundo Apellido        object
Nombre(s)               object
Nombre completo         object
Institución_origen      object
Rdoc                    object
Marca_temporal          object
Datos alternativos      object
dtype: object

Los números de `Edad` aparecen entre comillas ya que son texto.

In [6]:
df['Edad'].unique()

array(['55', nan, '19 semanas', '12 semanas', '31', '18', '39', '35',
       '74', '26', '36 semanas', '15 semanas', '83', '89', '64', '78',
       '73', '81', '75', '60', '63', '27', '45', '40', '38', '25', '90',
       '42', '77', '70', '47', '32', '20 semanas', '76', '84', '87', '66',
       '68', '94', '71', '92', '49', '69', '50', '80', '28', '37', '30',
       '20', '<17', '26 semanas', '34', '65', '86', '67', '58', '52',
       '85', '21 semanas', '17.6 semanas', '9 semanas', '33 semanas',
       '46', '22', '26.8 semanas', '22 semanas', '54', '33', '48', '53',
       '44', '57', '72', '41', '79', '29 semanas', '51', '82', '67-82',
       '88', '36', '96', '62', '91', '13 semanas', '5 semanas',
       '11 semanas', '1 mes', '25 semanas', '26-32 semanas', '14 semanas',
       '15', '40 semanas', '24 semanas', '24', '29', '13', '6',
       '23 semanas', '101', '93', '59', '19', '13.7 semanas', '61', '97',
       '86-87', '95', '17', '16', '23', '21', '18 semanas', '56',
       '30

Otra opción es convertir el tipo de dato columna por columna usando comandos como `.as_type` `.to_numeric` o `.to_datetime`.

![Función as_type](./images/pandas_astype.png)

Explicaremos esto con más detalle adelante.

## 3.2 Eliminación de observaciones y variables

Es posible que no todos los datos contenidos en el conjunto de datos sean de interes. Eliminarlos facilita el análisis y eficientiza el uso de memoria.

Podemos eliminar filas o columnas si:
* Baja variabilidad, por ejemplo una columna donde todos los valores son iguales
* Información repetida, por ejemplo filas idénticas
* Existe una gran cantidad de datos faltantes
* Información no relevante para el análisis.

Mucha de esta información se puede obtener del diccionario de datos y el profile.

Por ejemplo, en el subconjunto que estamos usando las columnas 'Estado_origen', 'Municipio_origen', 'Panteón_origen' tienen un solo valor, por lo que las eliminaremos

Podemos quitar las columnas usando el comando `.drop(axis=1)`. Si lo que queremos es quitar filas, se puede hacer de una manera similar, poniendo los nombres de las filas entre corchetes y usando el paramentro `axis=0`.

Para guardar estas modificaciones debemos de actualizar el dataframe

In [7]:
df = df.drop( ['Estado_origen', 'Municipio_origen', 'Panteón_origen'], axis=1)
df.head()

,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Rdoc,Marca_temporal,Datos alternativos
ID,,,,,,,,,,,,,,,,
XX-P001,Inhumación,2019-01-30 00:00:00,2018-03-04 00:00:00,NaN,Restos cremados,Masculino,55,Desconocido,NaN,NaN,NaN,,UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA,Sí,2020-03-23 00:00:00,NaN
XX-P002,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P003,Inhumación,2019-01-26 00:00:00,2018-12-13 00:00:00,NaN,Cadáver,Masculino,NaN,Conocido,Cruz,Lucero,Alberto,Alberto Cruz Lucero,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P004,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P005,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN


Para determinar filas duplicadas usamos el comando `duplicated()`.

In [8]:
df[ df.duplicated() ]

,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Rdoc,Marca_temporal,Datos alternativos
ID,,,,,,,,,,,,,,,,
XX-P010,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P011,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P016,Inhumación,2019-01-26 00:00:00,2018-12-23 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P032,Inhumación,2019-01-19 00:00:00,2018-12-04 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P041,Inhumación,2019-01-19 00:00:00,2018-12-09 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P043,Inhumación,2019-01-19 00:00:00,2018-12-09 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00,NaN
XX-P065,Inhumación,2019-01-12 00:00:00,2018-12-01 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-24 00:00:00,NaN
XX-P066,Inhumación,2019-01-12 00:00:00,2018-12-01 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-24 00:00:00,NaN
XX-P151,Inhumación,2018-11-24 00:00:00,2018-10-08 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-24 00:00:00,NaN


Podemos ver que hay un número de filas que tienen datos duplicados, es decir, que todos los datos se parecen. 
En este caso todos son `Desconocidos` y tienen identificadores distintos, por lo que lo mas probable es que no sean duplicados de verdad, sino un caso de registros indistinguibles por la falta de información.

En caso de querer quitarlos se pueden quitar usando el comando `drop_duplicates()`, el cual incluye opciones para determinar cual de los duplicados conservar.


Otra razón para quitar datos es por que hay una gran cantidad de faltantes. El comando `dropna()` sirve para quitar estas filas (`axis=0`) o columnas (`axis=1`).
Existen varias opciones:

* `how='any'` quita las filas o columnas donde hay al menos un nan, este parametró es el default
* `how='all'` quita las filas o columnas donde todos son nan
* `thresh=numero` quita las filas o columnas con más de cierto número de nans, esto es útil para detectar automáticamente columnas donde faltan muchos datos

Quitemos las columnas donde todos los valores son nan, en este caso 'Datos alternativos':

In [9]:
df = df.dropna( axis=1, how='all' )
df.head()

,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Rdoc,Marca_temporal
ID,,,,,,,,,,,,,,,
XX-P001,Inhumación,2019-01-30 00:00:00,2018-03-04 00:00:00,NaN,Restos cremados,Masculino,55,Desconocido,NaN,NaN,NaN,,UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA,Sí,2020-03-23 00:00:00
XX-P002,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00
XX-P003,Inhumación,2019-01-26 00:00:00,2018-12-13 00:00:00,NaN,Cadáver,Masculino,NaN,Conocido,Cruz,Lucero,Alberto,Alberto Cruz Lucero,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00
XX-P004,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00
XX-P005,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,NaN,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00


## 3.3 Valores faltantes

Un valor faltante es un valor que falta o está incompleto en un conjunto de datos. A menudo, los valores faltantes se deben a errores de entrada de datos, fallas en la recolección de datos o problemas técnicos. Los valores faltantes pueden aparecer como espacios en blanco, ceros, letras u otros caracteres. La presencia de valores faltantes en un conjunto de datos puede afectar la precisión y la calidad del análisis de los datos, por lo que es importante identificarlos y manejarlos adecuadamente durante la limpieza y el preprocesamiento de los datos.

Por ejemplo, la columna de 'Fecha_exhumación' tiene 99.9% de datos faltantes. 
Esta columna es un caso interesante, ya que contiene información del unico caso de exhumación de nuestro subconjunto de datos. Podríamos quitarla dada la cantidad de datos faltantes, pero antes de eso es importante ver la fila asociada.

In [10]:
df[ df['Fecha_exhumación'].notna() ]

,Estatus_FC,Fecha_inhumación,Fecha_defunción,Fecha_exhumación,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Rdoc,Marca_temporal
ID,,,,,,,,,,,,,,,
XX-P331,Exhumación,2019-03-02 00:00:00,2019-01-18 00:00:00,2020-03-31 00:00:00,Cadáver,Femenino,NaN,Conocido,Lopez,Gonzalez,Samara Judith,Samara Judith Lopez Gonzalez,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-27 18:16:02


En este caso es importante considerar que tipo de análisis nos interesa hacer y que información es necesaria.

Por ejemplo, si nos interesa las exhumaciones debemos de conservar la columna. Si nos interesan solo los cadaveres presentes en la fosa común se recomienda quitar la columna y la fila, ya que el cadaver ha sido removido. Sin embargo, si nos interesan las inhumanciones se debé de conservar la fila aunque se quite la columna, ya que el registro es tanto de una inhumación como de una exhumación. 
En general, es importante revisar los datos y tener claras las preguntas para tomar decisiones sobre la limpieza y análisis.

En este caso quitaremos las columnas con demasiados nan's, por ejemplo, dejaremos aquellas que tengan al menos 50 valores validos, lo cual debe de quitar 'Fecha_exhumación'

In [11]:
df = df.dropna(thresh=50, axis=1)
df.head()

,Estatus_FC,Fecha_inhumación,Fecha_defunción,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Rdoc,Marca_temporal
ID,,,,,,,,,,,,,,
XX-P001,Inhumación,2019-01-30 00:00:00,2018-03-04 00:00:00,Restos cremados,Masculino,55,Desconocido,NaN,NaN,NaN,,UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA,Sí,2020-03-23 00:00:00
XX-P002,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00
XX-P003,Inhumación,2019-01-26 00:00:00,2018-12-13 00:00:00,Cadáver,Masculino,NaN,Conocido,Cruz,Lucero,Alberto,Alberto Cruz Lucero,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00
XX-P004,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00
XX-P005,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00


Para determinar si debemos de quitar filas por nan en primer lugar hay que análizar la cantidad de faltantes y su distribución.

**Nota** Explicación del comando
```
df.isna()         <-- determinar si las celdas son na
  .sum(axis=1)    <-- sumar los faltantes por fila
  .value_counts() <-- contar cuantas veces aparece cada número
  .sort_index()   <-- ordenar por el indice, que en este caso es el número de faltantes
```


In [12]:
df.isna().sum(axis=1).value_counts().sort_index()

0    395
1    130
2     26
3    164
4    285
dtype: int64

Cómo podemos ver hay 285 filas donde faltan 4 valores, lo cual es poco menos de la mitad. Revisemos estas filas.

In [13]:
df[ df.isna().sum(axis=1)>=4 ]

,Estatus_FC,Fecha_inhumación,Fecha_defunción,Restos_tipo,Sexo,Edad,Conocido_Desconocido,Primer apellido,Segundo Apellido,Nombre(s),Nombre completo,Institución_origen,Rdoc,Marca_temporal
ID,,,,,,,,,,,,,,
XX-P002,Inhumación,2019-01-26 00:00:00,2018-12-12 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00
XX-P004,Inhumación,2019-01-26 00:00:00,2018-12-14 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00
XX-P005,Inhumación,2019-01-26 00:00:00,2018-12-18 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00
XX-P006,Inhumación,2019-01-26 00:00:00,2018-12-16 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00
XX-P008,Inhumación,2019-01-26 00:00:00,2019-12-17 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-03-23 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XX-P966,Inhumación,2018-07-14 00:00:00,2018-06-16 00:00:00,Cadáver,Masculino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-04-06 15:10:58.895000
XX-P967,Inhumación,2018-07-14 00:00:00,2017-08-08 00:00:00,Cadáver,Femenino,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-04-06 15:14:11.685000
XX-P974,Inhumación,2018-07-14 00:00:00,S/D,Restos humanos,Indeterminado,NaN,Desconocido,NaN,NaN,NaN,,INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...,Sí,2020-04-06 18:51:06.834000


3.3 Valores faltantes

En este 

En este caso son desconocidos donde no se sabe la edad, por lo que no es necesario quitarlas.

## Limpieza de texto

En este conjunto las columnas de texto son:
* 'Primer apellido'
* 'Segundo Apellido'
* 'Nombre(s)'
* 'Nombre completo'
* 'Rdoc'
* 'Datos alternativos'

Dependiendo del tipo de dato se pueden hacer diferentes operaciones. Por ejemplo, se puede pasar a mayusculas un texto pero no un número, se puede obtener el año de una fecha pero no de un texto

Por ejemplo, pensemos que queremos que las instituciones de origen no esten todas en mayusculas, sino solo la primera letra de la palabra.

Para hacer esto seleccionaremos la columna de interes, y luego aplicaremos la función `.str.title()`. 

**Nota**: existen varias funciones para mayusculas y minusculas como `.upper()`, `.lower()`, `.capitalize()` y `.title()`

In [3]:
df['Institución_origen'].str.title()

ID
XX-P001           Universidad Westhill - Facultad De Medicina
XX-P002     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P003     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P004     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P005     Instituto De Ciencias Forenses - Tribunal Supe...
                                  ...                        
XX-P996     Universidad Nacional Autónoma De México - Facu...
XX-P997     Universidad Nacional Autónoma De México - Facu...
XX-P998     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P999     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P1000    Instituto De Ciencias Forenses - Tribunal Supe...
Name: Institución_origen, Length: 1000, dtype: object

Podemos realizar más operaciones sobre esta selección, por ejemplo ordenar alfabeticamente obtener los valores únicos para obtener una lista de nombres propios.

In [4]:
df['Institución_origen'].str.title().sort_values().unique()

array(['Centro Cultural Universitario Justo Sierra',
       'Escuela De Medicina Saint Luke',
       'Institución De Asistencia Privada - Escuela Libre De Homeopatía De México',
       'Instituto De Ciencias Forenses - Tribunal Superior De Justicia De La Ciudad De México',
       'Instituto Politécnico Nacional - Escuela Nacional De Medicina Y Homeopatía',
       'Procuraduría General De La República',
       'Secretaría De La Defensa Nacional - Escuela Militar De Medicina',
       'The American British Cowdray Medical Center La.P.',
       'Universidad Anáhuac - Facultad De Ciencias De La Salud',
       'Universidad Nacional Autónoma De México - Facultad De Medicina',
       'Universidad Popular Autónoma Del Estado De Puebla',
       'Universidad Tominaga Nakamoto S.C. Escuela De Medicina Ciencias Básicas',
       'Universidad Westhill - Facultad De Medicina'], dtype=object)

Existen funciones que no se encuentran en pandas pero que pueden ser utiles. Por ejemplo, para ejemploquitar los acentos es posible con la función `unidecode`, la cual es parte de la biblioteca `unidecode`.

In [5]:
from unidecode import unidecode
unidecode('México')

'Mexico'

Podemos aplicar esta función a toda la columna

In [6]:
df['Institución_origen'].apply(unidecode)

ID
XX-P001           UNIVERSIDAD WESTHILL - FACULTAD DE MEDICINA
XX-P002     INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
XX-P003     INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
XX-P004     INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
XX-P005     INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
                                  ...                        
XX-P996     UNIVERSIDAD NACIONAL AUTONOMA DE MEXICO - FACU...
XX-P997     UNIVERSIDAD NACIONAL AUTONOMA DE MEXICO - FACU...
XX-P998     INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
XX-P999     INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
XX-P1000    INSTITUTO DE CIENCIAS FORENSES - TRIBUNAL SUPE...
Name: Institución_origen, Length: 1000, dtype: object

Además, podemos aplicar varias funciones juntas.

In [7]:
df['Institución_origen'].str.title().apply(unidecode)

ID
XX-P001           Universidad Westhill - Facultad De Medicina
XX-P002     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P003     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P004     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P005     Instituto De Ciencias Forenses - Tribunal Supe...
                                  ...                        
XX-P996     Universidad Nacional Autonoma De Mexico - Facu...
XX-P997     Universidad Nacional Autonoma De Mexico - Facu...
XX-P998     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P999     Instituto De Ciencias Forenses - Tribunal Supe...
XX-P1000    Instituto De Ciencias Forenses - Tribunal Supe...
Name: Institución_origen, Length: 1000, dtype: object

Sin embargo, al aplicar esta función a la columna `Nombre(s)` genera un error `AttributeError: 'float' object has no attribute 'encode'`. 

Esto se debe a que la columna incluye valores vacios, los cuales son de tipo `float`, por lo tanto la función `unidecode` se confunde, ya que los números no tienen acentos. 

Resolver este tipo de problemas es parte de la limpieza de datos y se verá mas adelante.

In [8]:
df['Nombre(s)'].apply( unidecode )

AttributeError: 'float' object has no attribute 'encode'

Dejo aqui una posible solución sin explicación (por el momento).

In [ ]:
df['Nombre(s)'].apply( lambda s: unidecode(s) if type(s)==str  else s )

### Categoria

En general se considera que un dato es categorico cuando hay pocas posibilidades definidas. Una aproximación rápida es que tenga menos de diez opciones de texto. Sin embargo esta no es una regla dura, por ejemplo el catálogo del INEGI de Estados, Municipios y Localidades tiene miles de opciones.

Podemos convertir una columna de tipo `object` a `category` con la función `.astype("category")`. 

En este ejemplo caso reescribiremos la columna para poder realizar varias operaciones.

In [ ]:
df['Restos_tipo'] = df['Restos_tipo'].astype("category")
df['Restos_tipo']

Podemos ver que el tipo a cambiado

In [ ]:
df.dtypes

Una ventaja de usar `category` es que permite definir un orden diferente al alfabético o numérico. Por ejemplo, definamos un orden especial para el tipo de restos.

In [ ]:
from pandas.api.types import CategoricalDtype

restos_categorias = 'Feto', 'Restos humanos', 'Miembros', 'Restos óseos', 'Restos cremados', 'Cadáver'
restos_categorias = CategoricalDtype(categories=restos_categorias, ordered=True)
df['Restos_tipo'] = df['Restos_tipo'].astype( restos_categorias )
df['Restos_tipo']

Ahora si ordenamos por categoria apareceran primero los Fetos y al final los Cadáveres.

In [ ]:
df.sort_values(by='Restos_tipo')

### Números

Existen dos tipos de datos númericos, enteros y flotantes. Los dos se comportan de forma muy similar.

En este conjunto de datos la columna `Edad` contiene numeros, sin embargo su tipo es `object` ya que hay edades como "18 semanas" que corresponden a fetos y neonatos.

In [ ]:
df['Edad'].str.upper()

In [ ]:
df['Fecha_inhumación'].dt.year

En caso de duda pandas asigna el tipo de dato object e infiere celda por celda el tipo de dato específico.

In [ ]:
df['Datos alternativos'].unique()

## 3.5 Tipos de datos

Cómo vimos en la sección anterior hay varios tipos de datos como: texto, númerico, fecha, etc.
De manera automática pandas infiere el tipo de datos al leer el archivo, de una manera muy similar a como Excel interpreta si un dato es texto o numérico.

Los tipos de datos de pandas son:
* `object`: texto o mezcla de varios tipos de datos, este es el tipo de dato por defecto
* `category`: información categórica, similar a Factors en R
* `int64`: números enteros
* `float64`: números con punto decimal
* `bool`: valores verdadero o falso
* `datetime64`: fechas
* `timedelta[ns]`: diferencias de tiempo

Podemos ver los tipos de datos inferidos usando la opción `.dtypes`.

In [ ]:
df.dtypes

### Texto

Dependiendo del tipo de dato se pueden hacer diferentes operaciones. Por ejemplo, se puede pasar a mayusculas un texto pero no un número, se puede obtener el año de una fecha pero no de un texto

Por ejemplo, pensemos que queremos que las instituciones de origen no esten todas en mayusculas, sino solo la primera letra de la palabra.

Para hacer esto seleccionaremos la columna de interes, y luego aplicaremos la función `.str.title()`. 

**Nota**: existen varias funciones para mayusculas y minusculas como `.upper()`, `.lower()`, `.capitalize()` y `.title()`

In [ ]:
df['Institución_origen'].str.title()

Podemos realizar más operaciones sobre esta selección, por ejemplo ordenar alfabeticamente obtener los valores únicos para obtener una lista de nombres propios.

In [ ]:
df['Institución_origen'].str.title().sort_values().unique()

Existen funciones que no se encuentran en pandas pero que pueden ser utiles. Por ejemplo, para ejemploquitar los acentos es posible con la función `unidecode`, la cual es parte de la biblioteca `unidecode`.

In [ ]:
from unidecode import unidecode
unidecode('México')

Podemos aplicar esta función a toda la columna

In [ ]:
df['Institución_origen'].apply(unidecode)

Además, podemos aplicar varias funciones juntas.

In [ ]:
df['Institución_origen'].str.title().apply(unidecode)

Sin embargo, al aplicar esta función a la columna `Nombre(s)` genera un error `AttributeError: 'float' object has no attribute 'encode'`. 

Esto se debe a que la columna incluye valores vacios, los cuales son de tipo `float`, por lo tanto la función `unidecode` se confunde, ya que los números no tienen acentos. 

Resolver este tipo de problemas es parte de la limpieza de datos y se verá mas adelante.

In [ ]:
df['Nombre(s)'].apply( unidecode )

Dejo aqui una posible solución sin explicación (por el momento).

In [ ]:
df['Nombre(s)'].apply( lambda s: unidecode(s) if type(s)==str  else s )

### Categoria

En general se considera que un dato es categorico cuando hay pocas posibilidades definidas. Una aproximación rápida es que tenga menos de diez opciones de texto. Sin embargo esta no es una regla dura, por ejemplo el catálogo del INEGI de Estados, Municipios y Localidades tiene miles de opciones.

Podemos convertir una columna de tipo `object` a `category` con la función `.astype("category")`. 

En este ejemplo caso reescribiremos la columna para poder realizar varias operaciones.

In [ ]:
df['Restos_tipo'] = df['Restos_tipo'].astype("category")
df['Restos_tipo']

Podemos ver que el tipo a cambiado

In [ ]:
df.dtypes

Una ventaja de usar `category` es que permite definir un orden diferente al alfabético o numérico. Por ejemplo, definamos un orden especial para el tipo de restos.

In [ ]:
from pandas.api.types import CategoricalDtype

restos_categorias = 'Feto', 'Restos humanos', 'Miembros', 'Restos óseos', 'Restos cremados', 'Cadáver'
restos_categorias = CategoricalDtype(categories=restos_categorias, ordered=True)
df['Restos_tipo'] = df['Restos_tipo'].astype( restos_categorias )
df['Restos_tipo']

Ahora si ordenamos por categoria apareceran primero los Fetos y al final los Cadáveres.

In [ ]:
df.sort_values(by='Restos_tipo')

### Números

Existen dos tipos de datos númericos, enteros y flotantes. Los dos se comportan de forma muy similar.

En este conjunto de datos la columna `Edad` contiene numeros, sin embargo su tipo es `object` ya que hay edades como "18 semanas" que corresponden a fetos y neonatos.

In [ ]:
df['Edad'].str.upper()

In [ ]:
df['Fecha_inhumación'].dt.year

En caso de duda pandas asigna el tipo de dato object e infiere celda por celda el tipo de dato específico.

In [ ]:
df['Datos alternativos'].unique()

In [ ]:
import pandas as pd
from numpy import nan

df_vic = pd.read_csv('data-raw/denuncias-victimas-pgj.csv')

### 3.4 Limpieza de datos

Generalmente, los conjuntos de datos contienen errores o incluyen datos que no son utiles para los análisis. Durante la limpieza de datos se detectan y corrigen datos incorrectos. Además, en este paso se quitan los datos que no se usaran en el análisis. 

En este caso haremos una serie de correciones:
* Quitar columnas
* Cambiar tipos de datos
* Modificar campos de texto
* Eliminar datos fuera de rango
* Datos faltantes
* Guardar datos

In [ ]:
df_vic = df_vic.drop(['idCarpeta', 'Año_hecho', 'Mes_hecho', 'ClasificacionDelito', 'lon', 'lat'], axis=1)
df_vic.columns = ['delito', 'categoria_delito', 'fecha_hechos', 
                  'Sexo', 'Edad', 'TipoPersona', 'CalidadJuridica', 'Geopoint']
df_vic.head()

#### Cambiar tipos de datos

Como ya vimos exiten varios tipos de datos. En partícular en este conjunto de datos, tenemos dos columnas de fechas, y si revisamos a que tipo de dato pertenecen usando el comando _dtype_ ,podemos observar que pandas las identificó como texto, lo cual dificulta trabajar con ellas. Si cambiamos estas columnas a tipo _datetime_, podemos usar las operaciones definidas para este tipo de datos. Haremos lo anterior empleando el comando _to_datetime_. Debido a que nos interesa guardar la transformación, es importante recordar hacer la asignación con _=_.

In [ ]:
df_vic['fecha_hechos'] = pd.to_datetime(df_vic['fecha_hechos'])

Usando este tipo de dato ahora es posible seleccionar por año, mes, día, hora y minuto usando los comandos:

* columna.dt.year
* columna.dt.month
* columna.dt.day
* columna.dt.hour
* columna.dt.minute

In [ ]:
df_vic['fecha_hechos'].dt.hour

Esto nos permite ver los delitos cometidos después de las 8pm. Para lograrlo usaremos _.dt.hour_ y el subsetting que ya hemos visto con anterioridad.

In [ ]:
df_vic[df_vic['fecha_hechos'].dt.hour>=20]

#### Modificar campos de texto

Cuando todas las columnas de texto están con mayúsculas, como en el grupo de datos con el que estamos trabajando, puede dificultarse la lectura. En este caso, vamos a cambiar el texto de mayúsculas de los delitos usando el comando _.capitalize()_, es decir este comando dejará la primera letra en mayúscula y todas las demás en minúscula. En el caso de las alcaldías y fiscalias usaremos _.title()_, con el fin de que la primera letra de todas las palabras sea mayúscula.

In [ ]:
df_vic['delito'] =           df_vic['delito'].str.capitalize()
df_vic['categoria_delito'] = df_vic['categoria_delito'].str.capitalize()
df_vic['TipoPersona'] =      df_vic['TipoPersona'].str.capitalize()
df_vic['CalidadJuridica'] =  df_vic['CalidadJuridica'].str.capitalize()
df_vic.tail()

#### Datos faltantes

Si revisamos los datos podemos ver que hay varios casos donde nos faltan datos, en especial en "Sexo" y "Edad".

In [ ]:
df_vic.count()

En el caso de "Sexo" podemos ver que en algunos casos se reporta que este "No se especifica" mientras que en otros se deja vacio, es decir, con NaN.

In [ ]:
df_vic['Sexo'].value_counts(dropna=False)

En este caso vamos a remplazar todos los "NaN" por "No se especifica" usando el comando _.fillna()_

In [ ]:
df_vic['Sexo'] = df_vic['Sexo'].fillna("No se especifica")
df_vic['Sexo'].value_counts(dropna=False)

#### Datos fuera de rango

__Fecha__

En el caso de este conjunto de datos tenemos las carpetas que se crearon entre 2016 y 2019. Sin embargo, esto no significa que todos los delitos hayan sido cometidos durante este periodo de tiempo. Para visualizarlo vamos a contar cuántas carpetas hay por año usando _.dt.year_ y _.value_counts()_.

In [ ]:
df_vic['fecha_hechos'].dt.year.value_counts()

En este caso podemos ver que hay carpetas sobre delitos cometidos en 1964. Para nuestro análisis vamos a quitar todas las carpetas que sean sobre hechos anteriores al 2016. Es decir, dejaremos unicamente las carpetas cuyo año sea mayor o igual a 2016.

In [ ]:
df_vic = df_vic[df_vic['fecha_hechos'].dt.year>=2016]

__Edad__

En el caso de la edad podemos recordar que hay una persona que tiene una edad de 258 años. Probablemente este sea un error de captura, ya que es la única persona reportada de mas de cién años. 
En este caso vamos a poner un críterio para manejar problemas similares a futuro, si la persona tiene mas de cién años remplazaremos su edad por nan. 
Para lograr esto seleccionaremos todas las celdas de personas de mas de cién años que están en la columna Edad con _.loc[]_ y remplazaremos estos valores por NaN. 
Esto debé de cambiar la edad máxima de nuestros datos.

In [ ]:
df_vic.loc[df_vic['Edad']>=100,'Edad'] = nan
df_vic['Edad'].max()

Otra cuestión que cabe destacar es que hay una gran cantidad de víctimas con edad de cero años. Este es un problema de la captura de los datos, ya que muchas veces, cuando no se conoce la edad de la víctima se pone un cero, lo cual se puede confundir con casos donde la víctima es un menor de edad. Sin embargo, en este caso es imposible corregir este error, por lo que dejaremos los datos como están, pero tendremos cuidado de esto durante el análisis.

In [ ]:
df_vic[df_vic['Edad']==0]

__Categorias y delitos__

En el pandas_profiling vimos que la gran mayoría de los delitos eran "de bajo impacto" o "hecho no delictivo". Quitar estos delitos nos permitiria simplificar el análisis, además de ocupar menos memoria. 

En primer lugar debemos revisar que tipo de delitos están en estas clasificaciones revisando los datos.

Para lograrlo:
* Encontrar todos los delitos que pertenezcan a las clasificaciones de interes usando _.isin()_
* Seleccionar las filas de interes y la columna de delitos con _.loc[]_
* Obtener los unicos con _.unique()_

In [ ]:
cat_interes = ["Delito de bajo impacto", "Hecho no delictivo"]
del_bajo_impacto = df_vic.loc[df_vic['categoria_delito'].isin(cat_interes), "delito"]
del_bajo_impacto.unique()

Para ver los delitos de "alto impacto" usaremos exactamente el mismo método, solo que agregaremos una negación _~_ al selector de filas.

In [ ]:
cat_interes = ["Delito de bajo impacto", "Hecho no delictivo"]
del_alto_impacto = df_vic.loc[~df_vic['categoria_delito'].isin(cat_interes), "delito"]
del_alto_impacto.unique()

Quitar los delitos de bajo impacto implica que nos concentramos en los que más afectan a la población, tales como: robo, secuestro y homicidio. No obstante, al mismo tiempo implica ignorar la forma de contacto con la violencia más común dentro de la población. La decisión de ignorar datos siempre debé de tomarse tomando en cuenta las implicaciones.

En este caso, para simplificar los análisis quitaremos los delitos de bajo impacto y los hechos no delictivos. 

Con la finalidad de quitar los delitos que no nos interesan aplicaremos una modificación del comando _isin()_. En el comando anterior vimos los delitos en la lista, para seleccionar los delitos que no están en la lista agregaremos _~_ al inicio del comando, lo cual representa una negación.

In [ ]:
df_vic = df_vic.loc[~df_vic['categoria_delito'].isin(['Delito de bajo impacto','Hecho no delictivo'])]

Después, de estas transformaciones nuestros datos son menos.

In [ ]:
df_vic.shape

__Tipo de persona y cálidad jurídica__

Una pregunta que podemos hacernos es si hay una relación entre el tipo de persona y la calidad juridica. 

Para verificar esto nos centraremos en análizar solo esas dos filas y usaremos el comando _.drop_duplicates()_, el cual quita todas las filas repetidas. En primer lugar seleccionaremos las dos columnas que nos interesan ('TipoPersona' y 'CalidadJuridica'), para después quitar todos los duplicados y comparar si siempre coinciden.

En este ocasión no vamos a guardar el resultado, solo veremos lo que sucede sin modificar los datos, y a partir de eso tomaremos una decisión.

In [ ]:
df_vic[['TipoPersona', 'CalidadJuridica']].drop_duplicates()

Podemos apreciar que no hay relación entre el tipo de persona y la calidad jurídica, por lo que ambas columnas nos proporcionan información relevante y dejaremos ambas. 
Este es un ejemplo de por que es importante revisar y entender los datos antes de modificarlos, ya que no siempre las transformaciones que hacemos durante el proceso de limpieza son adecuadas o necesarias.

Sin embargo, como podemos ver la Calidad Juridica tiene categorias que son equivalente como "Cadaver y Fallecido" o que podrían ser absorvidas en categorías mas grandes como "Menor víctima", "Victima  niño" y "Lesionado  adolescente". En este caso vamos a tratar de simplificar las categorias lo mas posible, en gran parte por que no todos los menores de edad vienen categorizados como "Menor víctima", lo cual puede causar errores en nuestros análisis posteriores.

In [ ]:
df_vic['CalidadJuridica'].value_counts()

El comando _replace_ toma un diccionario de python para hacer una lista de remplazo. Por ejemplo, para remplazar las categorías en las que dectamos error usaremos el siguiente dicionario de remplazo. Recuerda que los diccionarios usar _:_ para unir los pares de llave:valor y _,_ para separa los pares.

In [ ]:
remplazo = {"Fallecido":"Cadaver",
            "Menor víctima":"Victima",
            "Victima  niño":"Victima",
            "Lesionado  adolescente":"Lesionado"}
df_vic['CalidadJuridica'] = df_vic['CalidadJuridica'].replace(remplazo)

df_vic['CalidadJuridica'].value_counts()

Ahora las categorias juridicas que detectamos han sido sustituidas y llas víctimas se han sumado a sus nuevas categorias. Sin embargo, todas las mcategorias que no estaban en el diccionario de remplazo quedaron exactamente igual. Este metodo puede ser muy costoso cuando se quieren hacer muchos remplazos, ya que tendriamos que crear un diccionario con todas las opciones que deseamos remplazar. 

Una opción en esos casos es el comando _.map()_. El comando map remplaza solo los datos que están en el diccionario y llena todos aquellos que no están con NaN.

#### Guardar datos

Es posible guardar los datos limpios que hemos obtenido de varias formas, podemos guardarlos como csv o excel usando los comandos _.to_csv_ y _.to_excel_. Estos formatos tienen la ventaja de que son faciles de compartir. 

Nosotros guardaremos los datos limpios en la carpeta _data_clean_ como un csv, debido a que el tamaño de la base de datos es grande, puede ser complicado para manipularla en excel. Sin embargo, es posible abrir archivos csv usando excel.

Es muy importante no rescribir los datos originales y tratar de tener una carpeta para cada parte del proceso de análisis, para evitar perder información y poder reproducir confiablemente nuestros análisis.

In [ ]:
df_vic.to_csv('./data-clean/VictimasPgjCdmx-AltoImpacto.csv', index=False)

Una desventaja de guardar los archivos usando csv o excel, es que podemos perder el formato y los tipos de datos. Esto es importante sobretodo para tipos de datos como _datetime_. Una opción es guardar nuestros datos en un formato que sea facilmente interpretable para python, aunque este no se pueda trabajar con excel.

In [ ]:
from joblib import dump

with open('./data-clean/VictimasPgjCdmx-AltoImpacto.pkl', 'wb') as f:
    dump(df_vic, f)

#### Ejercicio 5

Revisa el archivo [CP-EjemploLimpiezaCarpetas](./extras/CP-EjemploLimpiezaCarpetas.ipynb) que se encuentra en la sección de extras. Este archivo realiza una limpieza similar para el conjunto de datos de las carpetas. Escribe que hace cada uno de los pasos y da las razones por las cuales se hacen.

In [ ]:

1. Escribe tres formas en las que te gustaría ordenar los datos.
2. Obten las tablas ordenadas, en caso de no poder hacerlo indica que datos te harían falta.